In [ ]:
using Graphs
using GraphPlot
using Combinatorics
using GraphIO
using Plots
using Statistics
using StatsBase
using JSON

function density(g)
    ne(g) / nv(g)
end

In [ ]:
graphs = loadgraphs("datasets/graphs.lg")

Distribution of size of graphs:

In [ ]:
histogram([nv(g) for (key, g) in graphs])

Choosing a k value:

In [ ]:
K = 15

# Preprocessing

In [ ]:
"Estimate the DamkS using the greedy approach"
function greedy_damks_estimate(g, k)
    h = deepcopy(g)
    # Remove vertices with minimum degree until the graph has `k` vertices left
    for c in 1:(nv(g) - k)
        min_d, min_v = findmin(v -> length(neighbors(h, v)), vertices(h))
        rem_vertex!(h, min_v)
    end
    # Remove vertices to check for better density
    max_d = density(h)
    best_g = deepcopy(h)
    for c in 1:min(k-1, nv(h))
        min_d, min_v = findmin(v -> length(neighbors(h, v)), vertices(h))
        rem_vertex!(h, min_v)
        if density(h) > max_d
            max_d = density(h)
            best_g = deepcopy(h)
        end
    end
    best_g
end

"Prune vertices that have degree lower than `d`"
function prune_graph(g, d)
    can_rem = true
    h = deepcopy(g)
    h_vmap = 1:nv(g)|>collect
    while can_rem
        to_be_rem = Vector{Int64}()
        for v in 1:nv(h)
            if length(neighbors(h, v)) < d
                push!(to_be_rem, v)
            end 
        end
        can_rem = length(to_be_rem) > 0
        h_vmap = map(x -> h_vmap[x], rem_vertices!(h, to_be_rem))
    end
    h, h_vmap
end

Pruning

In [ ]:
pruned = Dict(gn => Dict() for (gn, _) in graphs)
for (gn ,g) in graphs
    pruned[gn][:original] = g
    (h, h_vmap) = prune_graph(g, density(greedy_damks_estimate(g, K)))
    pruned[gn][:pruned] = h
    pruned[gn][:pruned_vmap] = h_vmap
end

Size distribution after pruning

In [ ]:
histogram([nv(ginfo[:pruned]) for (gn, ginfo) in pruned])

Select small graphs

In [ ]:
pruned = filter(((gn, ginfo),) -> nv(ginfo[:pruned]) <= 30, pruned)

# Densest At-most-k Subgraph

### 1. True DamkS

In [ ]:
function exact_daks(g, k)
    # Exact algorithm to find the densest at-k subgraph
    daks_vlist = nothing
    daks_density = 0
    for vec in combinations(1:nv(g), k)
        h, vmap = induced_subgraph(g, vec)
        if density(h) > daks_density
            daks_vlist = vmap
            daks_density = density(h)
        end
    end
    daks_vlist, daks_density
end

function exact_damks(g, k)
    # Exact algorithm to find the densest at-most-k subgraph
    damks_vlist = nothing
    damks_density = 0
    for ks in 2:k
        daks_g, daks_density = exact_daks(g, ks)
        if daks_density > damks_density
            damks_density = daks_density
            damks_vlist = daks_g
        end
    end
    damks_vlist, damks_density
end

function true_densities(graphs, k)
    # Exact algorithm to find the density of DamkS
    Dict(
        gn => exact_damks(ginfo[:pruned], k) for (gn, ginfo) in graphs
    )
end

function true_densities_mt(graphs, k)
    # Multi-threaded exact algorithm to find the density of DamkS
    gn_to_idx = Dict(gn => i for (i, (gn, _)) in enumerate(graphs))
    ts = [("", (Vector(), 0.0)) for _ in 1:length(graphs)]
    Threads.@threads for gn in graphs|>keys|>collect
        ts[gn_to_idx[gn]] = (gn, exact_damks(graphs[gn][:pruned], k))
    end
    Dict(gn => res for (gn, res) in ts)
end

Test on graph "1"

In [ ]:
# g = graphs["1"]
g = pruned["1"][:pruned]
# gplot(g)
damks_vlist, damks_density = exact_damks(g, K)
print("Density = ")
println(damks_density)
h, _ = induced_subgraph(g, damks_vlist)
gplot(h)

### 2. Run

Split into smaller chunks so we can save results while running.

In [ ]:
c = 10
ks = pruned|>keys|>collect
l = ks|>length
s = round(Int, l / c, RoundUp)
lst = [Dict(gn => pruned[gn] for gn in ks[((i-1)*s+1):min(i*s, l)]) for i in 1:c]

In [ ]:
for (i, split) in enumerate(lst)
    damks_res = true_densities_mt(split, K)
    js = Dict(gn => Dict(
        "size" => ginfo[:original]|>nv,
        "edges" => map(e -> (e|>src, e|>dst), ginfo[:original]|>edges|>collect),
        "damks_vlist" => map(v -> ginfo[:pruned_vmap][v], damks_res[gn][1]),
        "damks_density" => damks_res[gn][2],
    ) for (gn, ginfo) in split)
    open("outputs/split_$i.json", "w") do fp
        write(fp, json(js))
    end
end